In [ ]:
import torch
import numpy as np
from pytz import timezone
from datetime import datetime
import matplotlib.pyplot as plt
from stable_baselines3 import DDPG, SAC, TD3, PPO
from stable_baselines3.common.noise import NormalActionNoise

TZ = timezone('Europe/Moscow')

import sys
import os


sys.path.append(os.path.abspath('./env/'))
sys.path.append(os.path.abspath('./common/'))

from env_move_fast_sector import HumanMoveFastSectorAction
from env_move_sector_v2 import HumanMoveSectorActionV2
from env_move_sector_v3 import HumanMoveSectorActionV3
from env_move_ray_v3 import HumanMoveRayActionV3
from env_move_sector import HumanMoveSectorAction
from env_move_wall import HumanMoveAroundWallAction
from env_move_wall import HumanMoveAroundWallActionV2 
from env_move_wall import HumanMoveAroundWallActionV3
from env_move_wall import HumanMoveFastAroundWallAction 
from env_move_wall import HumanMoveRayAroundWallActionV3

device = "cuda" if torch.cuda.is_available() else "cpu"
device

In [ ]:

seed = int( datetime.now(TZ).strftime("%H%M%S") )

#env = HumanMoveRayActionV3( target_point_rand = False, object_ignore=False, tree_locate='wall7',tree_count=30, render_mode = 'human', seed=seed)
env = HumanMoveRayAroundWallActionV3( target_point_rand = True, object_ignore=False, object_locate='build10',wall_border=30,line_angle=-90, render_mode = 'human', seed=seed)
#env = HumanMoveFastAroundWallAction(target_point_rand=False, wall_border=5, tree_count=0, render_mode = 'human', seed=seed)
#env = HumanMoveAroundWallActionV3(target_point_rand=False, wall_border=35, tree_count=0, line_angle=90, tree_locate='wall2', render_mode = 'human', seed=seed)


model = PPO.load('env_Wall_MoveRay3_RanTP_exp_0112_115555/model.zip', env=env, device=device)
#model = SAC.load('env_MoveSector_exp_3110_174916/model.zip', env=env, device=device)

total_reward = 0.
step_reward = []
angle_step_reward = []
speed_step_reward = []
view_step_reward = []
stoper_step_reward = []
obstacle_stop_reward = []
obstacle_move_reward = []
a_speed_x = []
a_speed_y = []
a_speed_a = []

observation, info = env.reset(seed=seed)
# course move, speed
print(env.time_model_max)
min = int(env.time_model_max / 60)
sec = int(env.time_model_max) % 60
print(f'{min} min {sec} sec')

while True:

    action, _ = model.predict(observation)

    a_speed_x.append(action[0])
    a_speed_y.append(action[1])
    a_speed_a.append(action[2])

    observation, reward, terminated, truncated, info = env.step(action)
    if terminated or truncated:
        observation, info = env.reset()
        step_reward.append(reward)
        total_reward += reward
        print('BREAK')
        break
    
    step_reward.append(reward)
    total_reward += reward

    rews = env.get_rewards()
    angle_step_reward.append(rews['angle_reward'])
    speed_step_reward.append(rews['speed_reward'])
    view_step_reward.append(rews['view_reward'] )
    stoper_step_reward.append(rews['stoped_reward'])
    obstacle_stop_reward.append(rews['object_stop'])
    obstacle_move_reward.append(rews['object_move'])

print(len(step_reward))
print(total_reward)

In [ ]:
step_reward

In [14]:
env.close()

In [ ]:
#plt.plot(step_reward[:-5])
plt.plot(angle_step_reward,color='b')
plt.plot(speed_step_reward,color='g')
plt.plot(view_step_reward, color='pink')
plt.plot(stoper_step_reward, color='yellow')
plt.plot(obstacle_stop_reward,color='black')
plt.plot(obstacle_move_reward,color='r')
plt.legend([
    #'total',
    'angle',
    'speed',
    'view',
    'stoper',
    'obstacle_stop',
    'obstacle_free'
    ])
plt.title('Вознаграждения за эпизоды')
plt.xlabel('Эпизод')
plt.ylabel('Вознаграждения')
plt.show()

In [ ]:
plt.plot(step_reward[:-5])
plt.legend([
    'total',
    ])
plt.title('Вознаграждения за эпизоды')
plt.xlabel('Эпизод')
plt.ylabel('Вознаграждения')
plt.show()

In [ ]:
plt.plot(a_speed_x,color='b')
plt.plot(a_speed_y,color='g')
plt.plot(a_speed_a, color='r')
plt.legend([
    'speed_x',
    'speed_y',
    'angle',
    ])
plt.title('Вознаграждения за эпизоды')
plt.xlabel('Эпизод')
plt.ylabel('Вознаграждения')
plt.show()